<a href="https://colab.research.google.com/github/gcbirat12/All_Projects/blob/main/Customer_Purchases_Behaviour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd


In [13]:
data = pd.read_csv('/content/drive/My Drive/Customer_Purchases_Behaviour/customer_data.csv')
data.head()

,id,age,gender,income,education,region,loyalty_status,purchase_frequency,purchase_amount,product_category,promotion_usage,satisfaction_score
0,1,27,Male,40682,Bachelor,East,Gold,frequent,18249,Books,0,6
1,2,29,Male,15317,Masters,West,Regular,rare,4557,Clothing,1,6
2,3,37,Male,38849,Bachelor,West,Silver,rare,11822,Clothing,0,6
3,4,30,Male,11568,HighSchool,South,Regular,frequent,4098,Food,0,7
4,5,31,Female,46952,College,North,Regular,occasional,19685,Clothing,1,5


##Data Preprocessing

In [14]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define numerical and categorical columns
numerical_cols = ['age', 'income', 'promotion_usage', 'satisfaction_score']
categorical_cols = ['gender', 'education', 'region', 'loyalty_status', 'purchase_frequency', 'product_category']

# Create the preprocessing pipelines
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [15]:
# Assuming the target variable 'purchase_amount' has been isolated as y
X = data.drop(['id', 'purchase_amount'], axis=1)
y = data['purchase_amount']

# Apply preprocessing to the feature matrix
X_preprocessed = preprocessor.fit_transform(X)


##Model Training and Evaluation

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize and train the Linear Regression model
linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)

# Predict on the test set
y_pred = linear_reg_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}, MSE: {mse}, R^2: {r2}")


MAE: 1104.7260668445765, MSE: 2337716.3496762193, R^2: 0.8980247920781698


##Deployment

In [18]:
from joblib import dump

# Save the model to disk
dump(linear_reg_model, 'linear_reg_model.joblib')


['linear_reg_model.joblib']

In [27]:
from joblib import load

# Load the model from disk
loaded_model = load('linear_reg_model.joblib')


In [28]:
import pandas as pd
from joblib import load
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [30]:
# Assuming your model is saved as 'linear_reg_model.joblib'
model = load('linear_reg_model.joblib')


In [31]:
def predict_purchase_amount(customer_data):
    # Preprocess the input data
    customer_preprocessed = preprocessor.transform(customer_data)

    # Predict the purchase amount
    predicted_amount = model.predict(customer_preprocessed)

    return predicted_amount


In [32]:
def get_user_input():
    print("Please enter customer details for purchase amount prediction:")
    age = int(input("Enter age: "))
    gender = input("Enter gender (Male/Female): ")
    income = int(input("Enter annual income: "))
    education = input("Enter education level: ")
    region = input("Enter region: ")
    loyalty_status = input("Enter loyalty status: ")
    purchase_frequency = input("Enter purchase frequency (frequent/rare/occasional): ")
    product_category = input("Enter product category: ")
    promotion_usage = int(input("Enter promotion usage (0 for No, 1 for Yes): "))
    satisfaction_score = int(input("Enter satisfaction score (1-10): "))

    # Create a DataFrame with the input data
    customer_df = pd.DataFrame([[
        age, gender, income, education, region, loyalty_status, purchase_frequency,
        product_category, promotion_usage, satisfaction_score
    ]], columns=[
        'age', 'gender', 'income', 'education', 'region', 'loyalty_status',
        'purchase_frequency', 'product_category', 'promotion_usage', 'satisfaction_score'
    ])

    return customer_df


In [33]:
# Collect user input
customer_input_df = get_user_input()

# Predict purchase amount
predicted_amount = predict_purchase_amount(customer_input_df)

# Display the predicted purchase amount
print(f"Predicted Purchase Amount: ${predicted_amount[0]:,.2f}")


Please enter customer details for purchase amount prediction:
Enter age: 25
Enter gender (Male/Female): Male
Enter annual income: 77000
Enter education level: Bachelors
Enter region: East
Enter loyalty status: Gold
Enter purchase frequency (frequent/rare/occasional): rare
Enter product category: Food
Enter promotion usage (0 for No, 1 for Yes): 0
Enter satisfaction score (1-10): 7
Predicted Purchase Amount: $26,981.80
